In [2]:
import math
import sys
sys.path.insert(0, '../src')

from loaders import *
from episode import *
from dataset import *
from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [27]:
class CausalConv1d(nn.Module):
    # https://discuss.pytorch.org/t/causal-convolution/3456/4
    def __init__(self, in_channels, out_channels, kernel = 2,dilation=2):
        super(CausalConv1d, self).__init__()
        
        self.kernel = kernel
        self.padding = (kernel - 1)*dilation
        self.dilation = dilation
        self.causal_conv = nn.Conv1d(in_channels, out_channels, 2, padding = self.padding, dilation = dilation)

    def forward(self, input):
        return self.causal_conv(input)[:, :, :-self.padding]

In [57]:
class DenseBlock(nn.Module):
    
    def __init__(self,in_channels,num_filters,dilation_rate = 2):
        
        super(DenseBlock,self).__init__()
        
        self.causal_conv = CausalConv1d(in_channels,num_filters,2,dilation=dilation_rate)
        
    def forward(self,input):
        
        xf = F.tanh(self.causal_conv(input))
        xb = F.sigmoid(self.causal_conv(input))
        
        activations = xf*xb
        
        return torch.cat([input,activations],dim = 1)
        

In [66]:
class TCBlock(nn.Module):
    
    
    def __init__(self, in_channels, filters, seq_len):
        
        super(TCBlock, self).__init__()
        layer_count = math.ceil(math.log(seq_len)/math.log(2))
        blocks = []
        channel_count = in_channels
        for layer in range(layer_count):
            block = DenseBlock(channel_count, filters, dilation_rate=2**layer)
            blocks.append(block)
            channel_count += filters
        self.blocks = nn.Sequential(*blocks)

    def forward(self, input):
        return self.blocks(input)
    

In [79]:
class AttentionBlock(nn.Module):
    
    def __init__(self, dims, k_size, v_size, seq_len):
        
        super(AttentionBlock, self).__init__()
        
        self.key_layer = nn.Linear(dims, k_size)
        self.query_layer = nn.Linear(dims, k_size)
        self.value_layer = nn.Linear(dims, v_size)
        self.sqrt_k = math.sqrt(k_size)
        
    def forward(self,input):
        
        keys = self.key_layer(input)
        queries = self.query_layer(input)
        values = self.value_layer(input)
        
        logits = torch.bmm(queries, keys.transpose(2,1))
        
        mask = np.triu(np.ones(logits.size()), k=1).astype('uint8')
        mask = torch.from_numpy(mask)
        # do masked_fill_ on data rather than Variable because PyTorch doesn't
        # support masked_fill_ w/-inf directly on Variables for some reason.
        logits.data.masked_fill_(mask, float('-inf'))
        probs = F.softmax(logits, dim=1) / self.sqrt_k
        
        read = torch.bmm(probs, values)
        return torch.cat([input, read], dim=2)
        

In [9]:
eps = load_sampler_from_config("../src/test.yaml")
song = eps.get_episode().support[0][0]
song = list(song)

Blink 182 blowjob.mid
Blink 182 timetobreakup.mid
Blink 182 anewhope.mid
Blink 182 imlostwithoutyou.mid
Blink 182 depends.mid
Blink 182 boysdontcry.mid
Blink 182 dysenterygary.mid
Blink 182 dontleaveme.mid
Blink 182 toforgiveistosuffer.mid
Blink 182 countrysong(live).mid
Blink 182 familyreunion(versionlive).mid


In [60]:
dense_block = DenseBlock(1,5)

In [61]:
use_cuda = torch.cuda.is_available()
vocabulary_size = 16*128*2 + 32*16 + 100 + 1 # 4708 + 1
vocabulary_size = vocabulary_size + 2 # SOS (index 4709) and EOS (index 4710)
SOS_TOKEN = 4709
EOS_TOKEN = 4710
encoding_size = 500
one_hot_embeddings = np.eye(vocabulary_size)

In [62]:
input_variables = []
original_sequences = []

trunc_seq = [SOS_TOKEN] + list(song) + [EOS_TOKEN]
original_sequences.append(trunc_seq)
seq_length = len(trunc_seq)

trunc_seq = torch.from_numpy(np.array(one_hot_embeddings[trunc_seq])) # This is really time consuming

trunc_seq = trunc_seq.view(seq_length, vocabulary_size)
trunc_seq = Variable(trunc_seq).float()
input_variables.append(trunc_seq)

In [63]:
x = trunc_seq.unsqueeze(1)
x

Variable containing:
( 0  ,.,.) = 
   0   0   0  ...    0   1   0

( 1  ,.,.) = 
   0   0   0  ...    0   0   0

( 2  ,.,.) = 
   0   0   0  ...    0   0   0
 ... 

(499 ,.,.) = 
   0   0   0  ...    0   0   0

(500 ,.,.) = 
   0   0   0  ...    0   0   0

(501 ,.,.) = 
   0   0   0  ...    0   0   1
[torch.FloatTensor of size 502x1x4711]

In [64]:
dense_block(x)

Variable containing:
( 0  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  1.0000  0.0000
  0.1342  0.1342  0.1342  ...   0.1342  0.5171  0.1342
 -0.1723 -0.1723 -0.1723  ...  -0.1723 -0.0246 -0.1723
  0.3318  0.3318  0.3318  ...   0.3318  0.5119  0.3318
  0.1261  0.1261  0.1261  ...   0.1261  0.1798  0.1261
 -0.1673 -0.1673 -0.1673  ...  -0.1673 -0.1732 -0.1673

( 1  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.1342  0.1342  0.1342  ...   0.1342  0.1342  0.1342
 -0.1723 -0.1723 -0.1723  ...  -0.1723 -0.1723 -0.1723
  0.3318  0.3318  0.3318  ...   0.3318  0.3318  0.3318
  0.1261  0.1261  0.1261  ...   0.1261  0.1261  0.1261
 -0.1673 -0.1673 -0.1673  ...  -0.1673 -0.1673 -0.1673

( 2  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.1342  0.1342  0.1342  ...   0.1342  0.1342  0.1342
 -0.1723 -0.1723 -0.1723  ...  -0.1723 -0.1723 -0.1723
  0.3318  0.3318  0.3318  ...   0.3318  0.3318  0.3318
  0.1261  0.1261  0.1261  ...   0.1261  0.1261  0.1261


In [67]:
tc_block = TCBlock(1,5, 502)

In [68]:
tc_block(x)

Variable containing:
( 0  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  1.0000  0.0000
  0.0128  0.0128  0.0128  ...   0.0128  0.0120 -0.0275
 -0.0619 -0.0619 -0.0619  ...  -0.0619 -0.2063  0.0688
           ...             ⋱             ...          
  0.0370  0.0370  0.0347  ...   0.0261  0.0644  0.0306
  0.0293  0.0293  0.0303  ...   0.0325 -0.0120  0.0215
 -0.0101 -0.0101 -0.0090  ...  -0.0295 -0.0749 -0.0471

( 1  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0128  0.0128  0.0128  ...   0.0128  0.0128  0.0128
 -0.0619 -0.0619 -0.0619  ...  -0.0619 -0.0619 -0.0619
           ...             ⋱             ...          
  0.0370  0.0370  0.0347  ...   0.0261  0.0261  0.0261
  0.0293  0.0293  0.0303  ...   0.0325  0.0325  0.0325
 -0.0101 -0.0101 -0.0090  ...  -0.0295 -0.0295 -0.0295

( 2  ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0128  0.0128  0.0128  ...   0.0128  0.0128  0.0128
 -0.0619 -0.0619 -0.0619  ...  -0.0619 -0.0619 -0.0619


In [80]:
attention_block = AttentionBlock(4711, 300, 300, 502)

In [81]:
attention_block(x)

Variable containing:
( 0  ,.,.) = 
  0.0000  0.0000  0.0000  ...  -0.0005  0.0008  0.0001

( 1  ,.,.) = 
  0.0000  0.0000  0.0000  ...  -0.0001 -0.0005  0.0011

( 2  ,.,.) = 
  0.0000  0.0000  0.0000  ...  -0.0001 -0.0002 -0.0003
 ... 

(499 ,.,.) = 
  0.0000  0.0000  0.0000  ...  -0.0012  0.0005  0.0006

(500 ,.,.) = 
  0.0000  0.0000  0.0000  ...  -0.0012 -0.0008  0.0011

(501 ,.,.) = 
  0.0000  0.0000  0.0000  ...  -0.0010  0.0002  0.0008
[torch.FloatTensor of size 502x1x5011]